In [1]:
import numpy as np
from aocd import get_data, submit

In [2]:
test_input = "2333133121414131402"
real_input = get_data()

In [3]:
def parse_input(data):
    return np.array([int(x) for x in data])


In [4]:
test_data = parse_input(test_input)

In [5]:
0%2

0

In [6]:
def create_blocks(data):
    blocks = []
    for i, block in enumerate(data):
        if i%2 == 0:
            #data block
            blocks += [i//2 for _ in range(block)]
        else:
            blocks += [-1 for _ in range(block)]
    return blocks
        
        

In [7]:
create_blocks(parse_input("12345"))

[0, -1, -1, 1, 1, 1, -1, -1, -1, -1, 2, 2, 2, 2, 2]

In [12]:
def defragment(data):
    left_index = 0
    right_index = len(data)-1
    #once the indexes meet in the middle we are done
    while left_index < right_index:
        #next gap location
        while data[left_index] >= 0:
            left_index+=1
        #next file block locations
        while data[right_index] < 0:
            right_index-=1
        #dont proceed on last loop!!!
        if left_index >= right_index:
            break
        #swap places of the free space and file block
        data[left_index], data[right_index] = data[right_index], data[left_index]
    return data
    

In [13]:
data = defragment(create_blocks(parse_input(test_input)))

In [14]:
def checksum(defragmented_data):
    prods = [x*i for i, x in enumerate(defragmented_data) if x > 0]
    return np.sum(prods)
    

In [15]:
checksum(data)

np.int64(1928)

In [16]:
def run_defragmenter(puzzle_input):
    data = parse_input(puzzle_input)
    blocks = create_blocks(data)
    return(checksum(defragment(blocks)))

In [17]:
real_input = get_data()
part1 = run_defragmenter(real_input)

### Part two

In [253]:
def find_spaces(blocks):
    spaces = []
    start_index = None
    for i, block in enumerate(blocks):
        if block == -1:
            if start_index  is None:
                start_index = i
        else:
            if start_index is not None:
                space_len = i - start_index
                spaces.append((start_index, space_len))
                start_index = None
    if start_index is not None:
        space_len = len(blocks) - start_index
        spaces.append((start_index, space_len))
    return spaces


#why was this so hard to extend to all numbers from just the spaces...
def find_files(blocks):
    spaces = []
    start_index = None

    for i, block in enumerate(blocks):
        if block != -1: 
            if start_index is None:
                start_index = i
            elif blocks[start_index] != block:
                space_len = i - start_index
                spaces.append((start_index, space_len))
                start_index = i
        else:
            if start_index is not None:
                space_len = i - start_index
                spaces.append((start_index, space_len))
                start_index = None

    if start_index is not None:
        space_len = len(blocks) - start_index
        spaces.append((start_index, space_len))

    return spaces

In [260]:
def defragment2(blocks):
    spaces = find_spaces(blocks)
    files = find_files(blocks)
    # return

    for f in files[::-1]:
        file_ind, file_len = f
        # print(f"processing {f} of value {blocks[file_ind:file_ind+file_len]}")
        # print(spaces)
        for i, s in enumerate(spaces):
            # print(spaces)
            space_ind, space_len = s
            if space_ind >= file_ind:
                #no spaces to the left
                break
            if space_len >= file_len:
                # print(f"Found a suitable space of {space_len=} at loc {space_ind}")
                blocks[space_ind:space_ind+file_len], blocks[file_ind:file_ind+file_len] = blocks[file_ind:file_ind+file_len], blocks[space_ind:space_ind+file_len]
                # print(f'{"".join([str(x) if x >= 0 else "." for x in blocks])}')
                
                # correct_spaces = find_spaces(blocks)- #correct but sloooow

                #modifying spaces index on the go is faster than going to find spaces
                if  space_len - file_len > 0:
                    spaces[i] = (space_ind+file_len, space_len-file_len)
                else:
                    spaces.pop(i)
                # print(spaces, "\n", correct_spaces)
                
                break
        
    return blocks

In [261]:
test_data = parse_input(test_input)

test_blocks = create_blocks(test_data)

In [262]:
test_blocks = defragment2(test_blocks)

In [263]:
checksum(test_blocks)

np.int64(2858)

In [264]:
def run_defragmenter2(puzzle_input):
    data = parse_input(puzzle_input)
    blocks = create_blocks(data)
    return checksum(defragment2(blocks))

In [265]:
%%time
run_defragmenter2(real_input)

CPU times: user 94.2 ms, sys: 10.1 ms, total: 104 ms
Wall time: 104 ms


np.int64(6351801932670)